In [1]:
import os
import conda
import pandas as pd
import numpy as np
import json
from scipy.optimize import curve_fit
# from shapely.geometry import Polygon as Poly

pd.options.display.max_columns = 250

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

import matplotlib.pyplot as plt
import matplotlib.cm
import matplotlib.colors as colors

import geopandas
from geopandas.tools import sjoin
import geoplot as gplt
import geoplot.crs as gcrs

import folium
from folium.plugins import MousePosition, Search, FastMarkerCluster

import branca
import branca.colormap as cm

In [5]:
import folium
from folium.plugins import MousePosition, Search, FastMarkerCluster
import branca
import branca.colormap as cm

TSI_TOTAL_i = pd.read_csv('TSI_TOTAL_i.csv', converters={'HUC_8': lambda x: str(x)}).fillna(0)
poly  = geopandas.GeoDataFrame.from_file('watershed/huc8sum.shp')
poly['HUC_8'] = poly['HUC_8'].astype(str)
HUC8_json = poly.to_crs(epsg='4326').to_json()

# SoilGeoChemical_Pgrouped = pd.read_csv('SoilGeoChemical_Pgrouped.csv', converters={'HUC_8': lambda x: str(x)}).fillna(0)
TSI_TOTAL_GEOjson = poly.merge(TSI_TOTAL_i, on='HUC_8', how = 'left').fillna(np.nan)

m_TSI = folium.Map(tiles= 'openstreetmap', location=[48, -102], zoom_start=6)

# colormap1 = cm.linear.YlOrRd_09.scale(0,
#                                       TSI_TOTAL_i['TSI'].max())
# colormap1 = colormap1.to_step(index=[20,40,50,70,TSI_TOTAL_i['TSI'].max()])

colormap1 =cm.StepColormap(['#edf8e9','#bae4b3','#74c476','#238b45'], index=[20,40,50,70],
                vmin=TSI_TOTAL_i['TSI'].min(),
                vmax=TSI_TOTAL_i['TSI'].max())


TSI_TOTAL_dict = TSI_TOTAL_GEOjson.set_index('HUC_8')['TSI']

m_TSI_layer = folium.GeoJson(
    TSI_TOTAL_GEOjson.to_json(),
    name='Trophic State Index',
    style_function=lambda feature: {
        'fillColor': 'grey' if np.isnan(TSI_TOTAL_dict[feature['properties']['HUC_8']]) == True else colormap1(TSI_TOTAL_dict[feature['properties']['HUC_8']]),
        'fillOpacity': 0.9,
        'color': 'black',
        'weight': 0.9,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['HUC_8', 'TSI'],
        aliases=['HUC8 watershed', 'Trophic State Index'], 
        localize=True)
    
    ).add_to(m_TSI)

callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: "map-marker"});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""

folium.LayerControl().add_to(m_TSI)

formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

MousePosition(
    position='bottomright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(m_TSI)

HUC8search = Search(
    layer=m_TSI_layer,
    geom_type='Polygon',
    placeholder='Search for a HUC8 watershed',
    collapsed=False,
    search_label='HUC_8',
    weight=3
).add_to(m_TSI)

#Customized legend
legend_html =   '''  
                <div id='legend', 
                style="position: fixed; 
                            bottom: 50px; right: 50px; width: 250px; height: 170px; 
                            padding: 10px 10px;
                            border:2px solid grey; z-index:9999; font-size:16px;
                            background:#ffffff;
                            ">
                    <p><strong>Trophic State Index</strong></p>
                    <p>
                    <span style='color:#edf8e9'>▉</span> Oligotrophic<br>
                    <span style='color:#bae4b3'>▉</span> Mesotrophic<br>
                    <span style='color:#74c476'>▉</span> Eutrophic<br>
                    <span style='color:#238b45'>▉</span> Hypereutrophic<br>
                    <span style='color:#848889'>▉</span> No data available<br>
                    </p>
                </div>'
                ''' 
m_TSI.get_root().html.add_child(folium.Element(legend_html))

m_TSI.save('TSI_colormap.html')